# Scrape Data from Wikipedia

# --------------------------------------------------------------------------------------------------------
# PART 1: CREATE TORONTO DATASET
# --------------------------------------------------------------------------------------------------------

### Step 1: install packages

In [386]:
import pip

package_names=['geocoder','requests','html5lib','lxml','beautifulsoup4'] #packages to install
pip.main(['install'] + package_names + ['--upgrade']) 

!conda install -c conda-forge folium=0.5.0 --yes 
!conda install -c conda-forge geopy --yes 
print("")
print("***************************************************")
print("****************packages installed!****************")
print("***************************************************")
print("")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already up-to-date: geocoder in c:\users\harsh\anaconda3\lib\site-packages (1.38.1)
  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
Requirement already up-to-date: html5lib in c:\users\harsh\anaconda3\lib\site-packages (1.0.1)
Requirement already up-to-date: lxml in c:\users\harsh\anaconda3\lib\site-packages (4.5.0)
  Using cached beautifulsoup4-4.9.0-py3-none-any.whl (109 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Can't uninstall 'requests'. No files were found to uninstall.
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.8.2
    Can't uninstall 'beautifulsoup4'. No files were found to uninstall.
Solving environment: ...working... done

# All requested packages already installed.


***************************************************
****************packages installed!****************
***************************************************



usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


### Step 2: import libraries

In [384]:
from bs4 import BeautifulSoup # library to import data from a webpage
import requests #library to allow you to send HTTP requests using Python.
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # library for converting postcodes into long/lat geocodes
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library

### Step 3: import data from wikipedia

In [149]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.find('table', class_='wikitable sortable')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

### Step 4: read data into a dataframe and clean up

In [190]:
#instantiate dataframe
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

#initialise counters
col_counter = 0
row_counter = 0

#populate dataframe with wiki table values
for row in table.find_all('tr'):           #iterate along rows
    for cell in row.find_all('td'):        #iterate along columns
        col_counter += 1
        if col_counter == 1: 
            PC = cell.text.strip()         #assign PostalCode value to temporary variable
        if col_counter == 2: 
            BR = cell.text.strip()         #assign Borough value to temporary variable
            if BR == "Not assigned": 
                col_counter = 0            #re-initialise column counter, since we will jump to the first column of new row
                row_counter += 1           #increment row counter
                break                      #exit for loop and step into next row or 'tr' tag if Borough is unassigned
        if col_counter == 3:
            NH = BR                        #assign Borough value to temporary Neighborhood variable
            NH = cell.text.strip()         #overwrite Borough value with Neighborhood value (if not none)
            df.loc[row_counter] = PC,BR,NH #assign values from 3 temporary variables into row cells of dataframe
            col_counter = 0                #re-initialise column counter
            row_counter += 1               #increment row counter

#sort table by Postal Code, Borough, Neighborhood
df.sort_values(by=['PostalCode', 'Borough', 'Neighborhood'], inplace=True)

#replace "/" separators in Neighborhood to ","
df['Neighborhood'] = df['Neighborhood'].str.replace('/', ',' ) 

#reset index values to start from 0
df.reset_index(drop=True, inplace=True) 

### Step 5: Check table for duplicate Postal Codes and display table dimensions 

In [191]:
#Test for duplicate values in PostalCode
print("Are there unique values in PostalCode? ", df['PostalCode'].is_unique) 

#Output table dimensions (row, col)
print(df.shape)

Are there unique values in PostalCode?  True
(103, 3)


In [192]:
#Output table
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# --------------------------------------------------------------------------------------------------------
# PART 2: OBTAIN AND MERGE TORONTO DATA WITH LOCATION DATA
# --------------------------------------------------------------------------------------------------------

### Step 6: Obtain and merge geocode data

In [184]:
#TRIED THIS TECHNIQUE, BUT DOESNT RETURN VALUES, SO USING SUPPLIED CSV INSTEAD

    # initialize your variable to None
    #lat_lng_coords = None

    # loop until you get the coordinates
    #while(lat_lng_coords is None):
      #geo = geocoder.google('{}, Toronto, Ontario'.format('M1B'))
      #lat_lng_coords = geo.latlng

    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]

    #print("latitude,longitude")

geo = pd.read_csv('https://cocl.us/Geospatial_data')
geo.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 
print('Data downloaded!')
geo.head()

Data downloaded!


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [447]:
df = df.merge(geo, on='PostalCode', how='left', indicator=False)
df

,PostalCode,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029,43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577,43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848,43.692657,-79.264848


# --------------------------------------------------------------------------------------------------------
# PART 3: EXPLORE AND CLUSTER NEIGHBOURHOODS
# --------------------------------------------------------------------------------------------------------


### Step 7: Obtain venue data from neighborhoods in Toronto, and clean up data

In [341]:
CLIENT_ID = 'TKHE5SGMN2M4L55K4U1GEWEX4BK1ZSAO0JQCAFANS0OLSB2O' # your Foursquare ID
CLIENT_SECRET = 'W2HCCU0Q1NQTULOO4K41RMSV1DIJLHHCBAVFY43BC3ICS2BD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [342]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

#KeyError: 'groups' may mean exceeded free quota from FourSquare

KeyError: 'groups'

In [ ]:
# one hot encoding
toronto_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Venue_Cats = list(toronto_venues_onehot.columns)

# add neighborhood column back to dataframe
toronto_venues_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
toronto_venues_onehot = toronto_venues_onehot[['Neighbourhood'] + Venue_Cats]
toronto_venues_onehot.head()

In [244]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_venues_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.

In [431]:
# create list of columns "1st" to "10th Most Common Venue" venues
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new empty dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

# function to extract a temp dataframe with all columns and 1 row (row is oterated in function call 
# transpose the table, sort the venues in descending order, extract top 10 into an array
# where no data exists, pad the array out with NaN values.
# return the array to populate the table, row by row
def return_most_common_venues(row, num_top_venues):
    row.set_index('Neighbourhood',inplace=True)
    row = row.transpose() 
    row.columns.values[0] = 'rating'
    row = row[row['rating']>0]
    row_vals = row.index.values[0:num_top_venues]

    Loop_num = num_top_venues - len(row_vals)
    if Loop_num > 0:
        for x in range(Loop_num):
            row_vals = np.append(row_vals, [np.nan])

    return row_vals
        
# populate table
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[[ind]], num_top_venues)

toronto_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Lounge,Skating Rink,NaN,NaN,NaN,NaN,NaN,NaN
1,"Alderwood , Long Branch",Coffee Shop,Gym,Pharmacy,Pizza Place,Pub,Sandwich Place,Skating Rink,NaN,NaN,NaN
2,"Bathurst Manor , Wilson Heights , Downsview North",Bank,Bridal Shop,Chinese Restaurant,Coffee Shop,Deli / Bodega,Diner,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Ice Cream Shop
3,Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bedford Park , Lawrence Manor East",American Restaurant,Butcher,Café,Coffee Shop,Comfort Food Restaurant,Greek Restaurant,Grocery Store,Indian Restaurant,Italian Restaurant,Juice Bar
5,Berczy Park,Art Gallery,BBQ Joint,Bagel Shop,Bakery,Basketball Stadium,Beach,Beer Bar,Bistro,Breakfast Spot,Café
6,"Birch Cliff , Cliffside West",Café,College Stadium,General Entertainment,Skating Rink,NaN,NaN,NaN,NaN,NaN,NaN
7,"Brockton , Parkdale Village , Exhibition Place",Bakery,Bar,Breakfast Spot,Burrito Place,Café,Climbing Gym,Coffee Shop,Convenience Store,Furniture / Home Store,Grocery Store
8,Business reply mail Processing CentrE,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden,Garden Center,Gym / Fitness Center,Light Rail Station
9,"CN Tower , King and Spadina , Railway Lands , ...",Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Bar,Boat or Ferry,Boutique,Coffee Shop


### Step 8: Clustering neighborhoods

In [430]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0,
       0, 9, 0, 0, 8, 1, 0, 3, 0, 9, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 4,
       6, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 7, 0,
       0, 0, 0, 5, 7])

In [432]:
## create a new dataframe that includes the clusters
## as well as the top 10 venues for each neighborhood and long/lat.

# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#create index on Neighbourhood, prior to join
toronto_venues_sorted.set_index('Neighbourhood')

#rename column on df table, prior to join
df.rename(columns = {'Neighborhood':'Neighbourhood'}, inplace = True) 
toronto_merged = df.merge(toronto_venues_sorted, on='Neighbourhood', how='left', indicator=False)
toronto_merged.head() # check the last columns!

#Remove rows with NaN clusters - these exist where no venue data was obtained
print("No. of rows (including NaN clusters): ", toronto_merged.shape[0])
toronto_merged = toronto_merged[~toronto_merged['Cluster Labels'].isnull()]
print("No. of rows (excluding NaN clusters): ", toronto_merged.shape[0])
toronto_merged

No. of rows (including NaN clusters):  103
No. of rows (excluding NaN clusters):  98


,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,8.0,Fast Food Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,6.0,Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,0.0,Bank,Breakfast Spot,Electronics Store,Intersection,Medical Center,Mexican Restaurant,Rental Car Location,NaN,NaN,NaN
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Insurance Office,Korean Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Fried Chicken Joint,Gas Station,Hakka Restaurant,Thai Restaurant,NaN,NaN
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3.0,Convenience Store,Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029,0.0,Coffee Shop,Department Store,Discount Store,Hobby Shop,Train Station,NaN,NaN,NaN,NaN,NaN
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577,0.0,Bakery,Bus Line,Bus Station,Ice Cream Shop,Intersection,Metro Station,Park,Soccer Field,NaN,NaN
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476,0.0,American Restaurant,Motel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848,0.0,Café,College Stadium,General Entertainment,Skating Rink,NaN,NaN,NaN,NaN,NaN,NaN


### Step 9: Visualise clusters

In [440]:
latitude = 43.7869473
longitude = -79.385975

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Step 10: Examine Clusters

In [414]:
#Examine Cluster Number
clust_num = 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == clust_num, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,0.0,Bank,Convenience Store,Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,North York,0.0,Food & Drink Shop,Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,East York,0.0,Convenience Store,Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Central Toronto,0.0,Bus Line,Park,Swim School,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Downtown Toronto,0.0,Park,Playground,Trail,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,Central Toronto,0.0,Jewelry Store,Park,Sushi Restaurant,Trail,NaN,NaN,NaN,NaN,NaN,NaN
73,York,0.0,Field,Hockey Arena,Park,Trail,NaN,NaN,NaN,NaN,NaN,NaN
74,York,0.0,Park,Pool,Women's Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,North York,0.0,Bakery,Construction & Landscaping,Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Etobicoke,0.0,Park,Pool,River,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [441]:
#Examine Cluster Number
clust_num = 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == clust_num, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Bank,Breakfast Spot,Electronics Store,Intersection,Medical Center,Mexican Restaurant,Rental Car Location,NaN,NaN,NaN
3,Scarborough,0.0,Coffee Shop,Insurance Office,Korean Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Scarborough,0.0,Athletics & Sports,Bakery,Bank,Caribbean Restaurant,Fried Chicken Joint,Gas Station,Hakka Restaurant,Thai Restaurant,NaN,NaN
6,Scarborough,0.0,Coffee Shop,Department Store,Discount Store,Hobby Shop,Train Station,NaN,NaN,NaN,NaN,NaN
7,Scarborough,0.0,Bakery,Bus Line,Bus Station,Ice Cream Shop,Intersection,Metro Station,Park,Soccer Field,NaN,NaN
8,Scarborough,0.0,American Restaurant,Motel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Scarborough,0.0,Café,College Stadium,General Entertainment,Skating Rink,NaN,NaN,NaN,NaN,NaN,NaN
10,Scarborough,0.0,Chinese Restaurant,Indian Restaurant,Pet Store,Vietnamese Restaurant,NaN,NaN,NaN,NaN,NaN,NaN
11,Scarborough,0.0,Auto Garage,Bakery,Breakfast Spot,Middle Eastern Restaurant,Sandwich Place,Shopping Mall,Smoke Shop,NaN,NaN,NaN
12,Scarborough,0.0,Breakfast Spot,Latin American Restaurant,Lounge,Skating Rink,NaN,NaN,NaN,NaN,NaN,NaN


In [445]:
#Examine Cluster Number
clust_num = 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == clust_num, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1.0,Park,Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,North York,1.0,Food & Drink Shop,Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Downtown Toronto,1.0,Park,Playground,Trail,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,Central Toronto,1.0,Jewelry Store,Park,Sushi Restaurant,Trail,NaN,NaN,NaN,NaN,NaN,NaN
73,York,1.0,Field,Hockey Arena,Park,Trail,NaN,NaN,NaN,NaN,NaN,NaN
74,York,1.0,Park,Pool,Women's Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [444]:
#Examine Cluster Number
clust_num = 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == clust_num, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,2.0,Baseball Field,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,North York,2.0,Baseball Field,Food Service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [443]:
#Examine Cluster Number
clust_num = 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == clust_num, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3.0,Convenience Store,Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,Central Toronto,3.0,Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [442]:
#Examine Cluster Number
clust_num = 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == clust_num, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4.0,Garden,Ice Cream Shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
